初始化

In [1]:
import polars as pl
from polars import col
from datetime import datetime

### 如何选出指定日期的数据

构造数据

In [2]:
df = pl.DataFrame({
    "dates": ["2016-07-02", "2016-08-10",  "2016-08-31", "2016-09-10"],
    "values": [1, 2, 3, 4]
})
df

dates,values
str,i64
"""2016-07-02""",1
"""2016-08-10""",2
"""2016-08-31""",3
"""2016-09-10""",4


eager模式

In [3]:
# eager
# step1 :通过with_column对列进行变换
# step2 :使用filter进行条件筛选
(df.with_column(pl.col("dates").str.strptime(pl.Date)) 
 .filter(
     col("dates").is_between(datetime(2016, 8, 9), datetime(2016, 9, 1))
     )
)

dates,values
date,i64
2016-08-10,2
2016-08-31,3


或者用lazy模式

### 按另一列的值是否满足条件来对新增一列并赋值

In [5]:
df = pl.DataFrame({
    "a": [1, 2, 3, 4, 5],
    "b": list("abcde")
})
df

a,b
i64,str
1,"""a"""
2,"""b"""
3,"""c"""
4,"""d"""
5,"""e"""


In [6]:
df.with_column(
    pl.when(pl.col("a") > 3).then(
        10
    ).otherwise(
        pl.col("a")
    ).alias("new")
)

a,b,new
i64,str,i64
1,"""a""",1
2,"""b""",2
3,"""c""",3
4,"""d""",10
5,"""e""",10


### 缺失值判断(is_null)填补：用另一列，用均值等形式

In [7]:
# import polars as pl
# from polars import col

df = pl.DataFrame({
    "a": [0, 1, 2, 3, None, 5, 6, None, 8, None],
    "b": range(10),
})
df

a,b
i64,i64
0,0
1,1
2,2
3,3
null,4
5,5
6,6
null,7
8,8


In [8]:
out = df.with_columns([
    
    # 如果a为、null，则用对应行的 b 值进行填充
    pl.when(col("a").is_null()).then(
        col("b")).otherwise(
            col("a")).alias("a"),
    
    # 如果a为、null，则用对应行的上一行的 b 值进行填充
    # 且生成新的列，而不是在原列上填充
    pl.when(col("a").is_null()).then(
        col("b").shift(1)).otherwise(
            col("a")).alias("a_filled_lag"),
    
    # 均值填充
    pl.when(col("a").is_null()).then(
        col("b").mean()).otherwise(
            col("a")).alias("a_filled_mean")

])
out

a,b,a_filled_lag,a_filled_mean
i64,i64,i64,f64
0,0,0,0.0
1,1,1,1.0
2,2,2,2.0
3,3,3,3.0
4,4,3,4.5
5,5,5,5.0
6,6,6,6.0
7,7,6,4.5
8,8,8,8.0


### 列的描述性统计

In [9]:
df = pl.DataFrame({
    'a': [1.0, 2.8, 3.0],
    'b': [4, 5, 6],
    "c": [True, False, True]
    })
df.describe()
# df.describe().transpose()

describe,a,b,c
str,f64,f64,f64
"""mean""",2.266667,5.0,null
"""std""",1.101514,1.0,null
"""min""",1.0,4.0,0.0
"""max""",3.0,6.0,1.0
"""median""",2.8,5.0,null


### 选取最小值

In [10]:
# import polars as pl

df = pl.DataFrame({
    "a": [1,4,2,5,2],
    "b": [5,1,4,2,5],
    "c": [3,2,5,7,2]
})
df

a,b,c
i64,i64,i64
1,5,3
4,1,2
2,4,5
5,2,7
2,5,2


返回series

In [11]:
df.min(axis=1)

shape: (5,)
Series: 'a' [i64]
[
	1
	1
	2
	2
	2
]

返回pl.DF

In [12]:
df.select(
    df.min(axis=1).alias('min')
)

min
i64
1
1
2
2
2


或者

In [13]:
df.select(
    pl.min(["a", "b", "c"]) 
)

min
i64
1
1
2
2
2


或者
https://stackoverflow.com/questions/69888178/calculating-rowwise-minimum-of-two-series

In [14]:
df.select(
    pl.fold(int(1e9), 
            lambda acc, a: pl.when(acc > a).then(a).otherwise(acc), ["a", "b", "c"]
    )
)

literal
i64
1
1
2
2
2


### 取出一个Series

In [15]:
df = pl.DataFrame({
    "A": [1, 2, 3],
    "B": [1, 2, 3]
})

assert isinstance(df["A"], pl.Series)

### 生成各种形式的滞后项

In [16]:
# some initial dataframe
df = pl.DataFrame({
    "a": [1, 2, 3, 4, 5],
    "b": [5, 4, 3, 2, 1]
})
df

a,b
i64,i64
1,5
2,4
3,3
4,2
5,1


In [17]:
# a function that returns a lazy evaluated expression
def lag(name: str, n: int) -> pl.Expr:
    return pl.col(name).shift(n).suffix(f"_lag_{n}")

# a lazy evaluated expression assigned to a variable
lag_foo = lag("a", 1)

out = df.select(
    [lag_foo,] +
    [lag("b", i) for i in range(5)]  # create exprs with a list comprehension
)
out

a_lag_1,b_lag_0,b_lag_1,b_lag_2,b_lag_3,b_lag_4
i64,i64,i64,i64,i64,i64
null,5,null,null,null,null
1,4,5,null,null,null
2,3,4,5,null,null
3,2,3,4,5,null
4,1,2,3,4,5


### 统计出现的频率并重复这个频率到每个值

In [18]:
# import polars as pl

df = pl.DataFrame({"md5": ["a", "a", "b","c","c","a"]})
out_df = df.select([
    pl.col("*"),
    pl.col("md5").count().over("md5").alias("row_count"),
])

out_df

md5,row_count
str,u32
"""a""",3
"""a""",3
"""b""",1
"""c""",2
"""c""",2
"""a""",3


比如我们新增一列

In [19]:
# import polars as pl

df = pl.DataFrame({"md5": [123, 123, 333,666,666,123]})
out_df = df.select([
    pl.col("*"),
    pl.col("md5").count().over("md5").alias("row_count"),
]).with_column(
    # 出现的频数越高，对其按权重扩大
    (pl.col("md5") * pl.col("row_count")).alias("weighted")
)

out_df

md5,row_count,weighted
i64,u32,i64
123,3,369
123,3,369
333,1,333
666,2,1332
666,2,1332
123,3,369


In [20]:
df = pl.DataFrame({"md5": ["a", "a", "b","c","c","a"]})
out_df = df.groupby(
    "md5"
).count()
out_df

md5,count
str,u32
"""a""",3
"""c""",2
"""b""",1


### 判断某列包含某个字符串的那些行

In [21]:
# import polars as pl

df = pl.DataFrame({"a": [1, 2, 3], "b": ["hello", "world", "everyone"]})
df

a,b
i64,str
1,"""hello"""
2,"""world"""
3,"""everyone"""


In [22]:
search = "hello"

df["b"].str.contains(search)  # this works

# df["a"].str.contains(search)  # 因为a is not of type Utf8
df["a"].cast(pl.Utf8).str.contains(search)  # this works

shape: (3,)
Series: 'a' [bool]
[
	false
	false
	false
]

### arr属性

In [23]:
df = pl.DataFrame({"my_column": [[1,2,3],[1,2,3],[1,2,3]]})
df

my_column
list
"[1, 2, 3]"
"[1, 2, 3]"
"[1, 2, 3]"


In [24]:
df.with_column(pl.col("my_column").arr.concat([4, 5]))

my_column
list
"[1, 2, ... 5]"
"[1, 2, ... 5]"
"[1, 2, ... 5]"


### 与pandas中的merge对应


```python
# pandas 
# CPU times: user 1.64 s, sys: 867 ms, total: 2.5 s
# Wall time: 2.52 s
df1.merge(df2, left_on="a", right_on="b")

# polars
# CPU times: user 5.59 s, sys: 199 ms, total: 5.79 s
# Wall time: 780 ms
df1p.join(df2p, left_on="a", right_on="b")
```

### 类似于pandas中的transform函数是over
#### 新增一列，内容为每个组，组内的序号，使用cumcount函数

Get an array with the cumulative count computed at every element.   
Counting from 0 to len

In [25]:
day = ['day1','day2','day3','day4','day1','day2','day3','day1','day2']
code = ["a","a","a","a","b","b","b","c","c"]
price = [1,2,3,4,5,6,7,8,9]
df = pl.DataFrame({"date":day,"code":code,"price":price})
df

date,code,price
str,str,i64
"""day1""","""a""",1
"""day2""","""a""",2
"""day3""","""a""",3
"""day4""","""a""",4
"""day1""","""b""",5
"""day2""","""b""",6
"""day3""","""b""",7
"""day1""","""c""",8
"""day2""","""c""",9


In [26]:
(df.select([
    pl.all(),
    # 即按code分组
    # 然后通过pl.col("date").cumcount()为每组生成序号
    pl.col("date").cumcount().over("code").alias("codeindex"),
]))

date,code,price,codeindex
str,str,i64,u32
"""day1""","""a""",1,0
"""day2""","""a""",2,1
"""day3""","""a""",3,2
"""day4""","""a""",4,3
"""day1""","""b""",5,0
"""day2""","""b""",6,1
"""day3""","""b""",7,2
"""day1""","""c""",8,0
"""day2""","""c""",9,1


其实把date换做code也是可以的

In [27]:
(df.select([
    pl.all(),
    pl.col("code").cumcount().over("code").alias("codeindex"),
]))

date,code,price,codeindex
str,str,i64,u32
"""day1""","""a""",1,0
"""day2""","""a""",2,1
"""day3""","""a""",3,2
"""day4""","""a""",4,3
"""day1""","""b""",5,0
"""day2""","""b""",6,1
"""day3""","""b""",7,2
"""day1""","""c""",8,0
"""day2""","""c""",9,1


### 判断两个pl.DF是否相同

In [28]:
# import polars as pl
pl.DataFrame({"x": [1,2,3]}).frame_equal(
    pl.DataFrame({"x": [1,2,3]}))  # True

True

### 拆分并统计句子中的词频

In [29]:
csv = """
0,                            Would never order again.
1,   I'm not sure it gives me any type of glow and ...
2,   Goes on smoothly a bit sticky and color is clo...
3,       Preferisco altri prodotti della stessa marca.
4,        The moisturizing advertised is non-existent.
""".encode()

df = pl.read_csv(csv, has_header=False, new_columns=["idx", "lines"])
df

idx,lines
i64,str
0,""" Would never order again."""
1,""" I'm not sure it gives me any type of glow and ..."""
2,""" Goes on smoothly a bit sticky and color is clo..."""
3,""" Preferisco altri prodotti della stessa marca."""
4,""" The moisturizing advertised is non-existent."""


In [30]:
df.select(pl.col("lines").str.split(" "))
# 注意，str.split(" ")中还包含了空格的

lines
list
"["""", """", ... ""again.""]"
"["""", """", ... ""...""]"
"["""", """", ... ""clo...""]"
"["""", """", ... ""marca.""]"
"["""", """", ... ""non-existent.""]"


In [31]:
df.select( pl.col("lines").str.split(" ").flatten() )
# 拉平

lines
str
""""""
""""""
""""""
""""""
""""""
""""""
""""""
""""""
""""""


In [32]:

(
    df.select( pl.col("lines").str.split(" ").flatten().alias("words"))
    .groupby("words").agg(pl.count())
    .sort("count", reverse=True)
    .filter(pl.col("words").str.lengths() > 0)  
)


words,count
str,u32
"""is""",2
"""and""",2
"""sticky""",1
"""Goes""",1
"""The""",1
"""smoothly""",1
"""advertised""",1
"""stessa""",1
"""any""",1


### 先分组，如等于组内最值，则取出

In [33]:
df = pl.DataFrame(dict(
    x=[0, 0, 1, 1], 
    y=[1, 2, 3, 3])
)
df

x,y
i64,i64
0,1
0,2
1,3
1,3


In [34]:
# 按x分组后，y的两个组内最大值为2和3
# 当y取这两个值的时候，选出
df.filter(
    (  pl.col("y") == pl.max("y").over("x") )
)

x,y
i64,i64
0,2
1,3
1,3


### 判断出否为重复值

In [35]:
df = pl.DataFrame(
    {
        "a": [1, 2, 3, 1, 3],
        "b": ["x", "y", "z", "x","s"],
    }
)
df

a,b
i64,str
1,"""x"""
2,"""y"""
3,"""z"""
1,"""x"""
3,"""s"""


### 删除重复值

In [36]:
df.is_duplicated()

shape: (5,)
Series: '' [bool]
[
	true
	false
	false
	true
	false
]

In [37]:
df.distinct(subset=["a","b"])

a,b
i64,str
1,"""x"""
2,"""y"""
3,"""z"""
3,"""s"""


In [38]:
df.distinct(subset=["a"])

a,b
i64,str
1,"""x"""
2,"""y"""
3,"""z"""


In [39]:
df = pl.DataFrame({
    "groups": ["a"] * 10 + ["b"] * 20,
    "values": range(30)
})
df

groups,values
str,i64
"""a""",0
"""a""",1
"""a""",2
"""a""",3
"""a""",4
"""a""",5
"""a""",6
"""a""",7
"""a""",8


### 获取分组的前20%

我们可以使用 head(10) 来获取 col 中的前 10 个元素。  
但是，如果组的长度不同，我需要在每个组中获取前 20% 的元素，  
例如 10 个元素组中的 2 个元素。  
例如 20 个元素组中的 4 个元素。 

如果我们的代码这么写，得到的结果不是我们想要的

In [40]:
(df.groupby("groups")
    .agg( pl.all().head(pl.count() * 0.2) )
)

groups,values
str,list
"""b""","[10, 11, ... 13]"
"""a""","[0, 1]"


所以要explode

In [41]:
(df.groupby("groups")
    .agg( pl.all().head(pl.count() * 0.2) )
    .explode( pl.all().exclude("groups") )
)

groups,values
str,i64
"""b""",10
"""b""",11
"""b""",12
"""b""",13
"""a""",0
"""a""",1


```python
exclude("groups")
```
其实就是这一部分不展开，但是会在values展开后被自动广播

那么来学一下explode函数，explode前面其实是被展开的那个列

In [42]:
df = pl.DataFrame({"b": [[1, 2, 3], [4, 5, 6]]})
df

b
list
"[1, 2, 3]"
"[4, 5, 6]"


In [43]:
df.select(pl.col("b").explode())

b
i64
1
2
3
4
5
6


### 新建了列，固定值

In [44]:
df = pl.DataFrame(
    {
        "a": [1, 2, 3, 1, 3],
        "b": ["x", "y", "z", "x","s"],
    }
)
df
df.with_column(pl.lit(6).alias('VERSION'))

a,b,VERSION
i64,str,i32
1,"""x""",6
2,"""y""",6
3,"""z""",6
1,"""x""",6
3,"""s""",6


### pl.DF与pandas.DF的互转

#### pl.DF  to pd.DF

In [45]:
df = pl.DataFrame(
    {
        "a": [1, 2, 3, 1, 3],
        "b": ["x", "y", "z", "x","s"],
    }
)
dfp = df.to_pandas()
# 转为pandas

#### pd.DF to pl.DF

In [46]:
print('type is {}'.format(type(pl.DataFrame(dfp))))

type is <class 'polars.internals.frame.DataFrame'>


#### pl.Series to pd.Series

In [47]:
pds = pl.Series([1, 2, 3]).to_pandas()
pds

0    1
1    2
2    3
dtype: int64

#### pl.Series to pd.Series

In [48]:
pl.Series(pds)

shape: (3,)
Series: '' [i64]
[
	1
	2
	3
]

### 批量修改列名

In [49]:
df = pl.DataFrame({
    "a.c": [1, 2],
    "b.d": [3, 4]
})
df.columns = list(map(lambda x: x.replace(".", "_"), df.columns))
df

a_c,b_d
i64,i64
1,3
2,4


### 列名排序

In [50]:
df = pl.DataFrame({
    "c": [1, 2],
    "a": ["a", "b"],
    "b": [True, False]
})

df.select(sorted(df.columns))

a,b,c
str,bool,i64
"""a""",true,1
"""b""",false,2


### 分组后按组内索引取到对应的行

In [51]:
df = pl.DataFrame(dict(x=[1,0,1,0,1,0], y=[1,2,3,4,5,6]))
df

x,y
i64,i64
1,1
0,2
1,3
0,4
1,5
0,6


下面的代码表示取到组内索引为0和2的行

In [52]:
df.groupby("x").agg(pl.all().take([0, 2]))

x,y
i64,list
1,"[1, 5]"
0,"[2, 6]"


In [53]:
import polars as pl

df = pl.DataFrame(
    {"version": [9, 85, 87], "testsuite": ["scan1", "scan2", "scan3"], "status": ["ok"] * 3})
df

version,testsuite,status
i64,str,str
9,"""scan1""","""ok"""
85,"""scan2""","""ok"""
87,"""scan3""","""ok"""


In [54]:
wide = df.pivot(index="testsuite", 
                columns='version', 
                values='status',
                maintain_order=False)
wide

testsuite,85,87,9
str,str,str,str
"""scan1""",null,null,"""ok"""
"""scan2""","""ok""",null,null
"""scan3""",null,"""ok""",null


In [55]:
# df["version"].to_list()
cols = df["version"].cast(pl.Utf8).to_list()
cols

['9', '85', '87']

In [56]:
wide[ ["testsuite"] + cols ]
# 把 cols 的每个值作为列

testsuite,9,85,87
str,str,str,str
"""scan1""","""ok""",null,null
"""scan2""",null,"""ok""",null
"""scan3""",null,null,"""ok"""


In [58]:
df = pl.DataFrame(dict(x=[1,1,0,0], y=[1,2,3,4]))
df

x,y
i64,i64
1,1
1,2
0,3
0,4


In [64]:
df.select(
     pl.first().len()
)

x
u32
4


### 使用alias复制和使用rename重命名列

In [65]:
k2 = pl.DataFrame({"column1": [1,2,3],
                   "column2": [4,5,6],
                   "column3": [7,8,9],
                   "column4": [10,11,12]})
k2

column1,column2,column3,column4
i64,i64,i64,i64
1,4,7,10
2,5,8,11
3,6,9,12


In [68]:
# 其实都变成了新增
k2.with_columns([pl.col("column1").alias("new column 1"), 
                 pl.col("column4").alias("column5")])

column1,column2,column3,column4,new column 1,column5
i64,i64,i64,i64,i64,i64
1,4,7,10,1,10
2,5,8,11,2,11
3,6,9,12,3,12


In [70]:
k2.rename(
    {'column1':'new column 1'}
).with_columns( [pl.col("column4").alias("column5")])

new column 1,column2,column3,column4,column5
i64,i64,i64,i64,i64
1,4,7,10,10
2,5,8,11,11
3,6,9,12,12


### 用alisa交换两列

In [71]:
# 其实都变成了新增
k2.with_columns([pl.col("column1").alias("column3"), 
                 pl.col("column3").alias("column1")])

column1,column2,column3,column4
i64,i64,i64,i64
7,4,1,10
8,5,2,11
9,6,3,12


### 通过parse_dates参数，在读入时自动解析日期格式

In [73]:
from io import StringIO

my_csv = StringIO(
"""
ID,start,last_updt,end
1,2008-10-31, 2020-11-28 12:48:53,12/31/2008
2,2007-10-31, 2021-11-29 01:37:20,12/31/2007
3,2006-10-31, 2021-11-30 23:22:05,12/31/2006
"""
)

pl.read_csv(my_csv, parse_dates=True)

ID,start,last_updt,end
i64,date,datetime,str
1,2008-10-31,2020-11-28 12:48:53,"""12/31/2008"""
2,2007-10-31,null,"""12/31/2007"""
3,2006-10-31,null,"""12/31/2006"""


### 使用repeat和explode对agg进行广播

In [78]:
df = pl.DataFrame(dict(x=[1,2,3]))
df

x
i64
1
2
3


In [80]:
df.with_column(pl.lit(1).alias("y"))

x,y
i64,i32
1,1
2,1
3,1


In [87]:

df = pl.DataFrame(dict(x=[1,1,0,0])).groupby("x")
df.agg( pl.repeat(1, pl.count() ) ).explode(
    pl.col('literal')
)


x,literal
i64,i32
0,1
0,1
1,1
1,1


### 一个复杂的应用场景
https://stackoverflow.com/questions/71552672/how-to-filter-record-sequences-from-a-polars-dataframe-using-multiple-threads

应用场景  
首先我们有各个时期，本来每个时期对每个用户都应该有记录的  
但是部分id中间断了，只有它在某个时期没有记录，那么关于这个用户的后面的记录都删掉

In [88]:
df = pl.DataFrame({'Id': [1,1,2,2,2,2,3,3,4,4,4,5,5,5,6,6,6],
 'Age': [1,4,1,2,3,4,1,2,1,2,3,1,2,4,2,3,4],
 'Value': [1,142,4,73,109,145,6,72,-8,67,102,-1,72,150,72,111,149]})
df

Id,Age,Value
i64,i64,i64
1,1,1
1,4,142
2,1,4
2,2,73
2,3,109
2,4,145
3,1,6
3,2,72
4,1,-8


In [93]:
df2 = df.filter(
    pl.col('Age').rank().over('Id') == pl.col('Age')
)
df2

Id,Age,Value
i64,i64,i64
1,1,1
2,1,4
2,2,73
2,3,109
2,4,145
3,1,6
3,2,72
4,1,-8
4,2,67


尝试一点点来解析

In [100]:
# 首先来看一下age被排序后的情况
df.select(
    pl.col('Age').rank()
)
# 我们可以看到，所有的值被计算了值

Age
f32
3.0
15.5
3.0
8.0
12.0
15.5
3.0
8.0
3.0


In [101]:
# 然后我们对秩进行了分组，按照id分的
df.select(
    pl.col('Age').rank().over('Id')
)
# 如果我分组后的值，跟Age的值一样，那就说明没有缺失

Age
f32
1.0
2.0
1.0
2.0
3.0
4.0
1.0
2.0
1.0


那么问题来了，如果我们这里的age，不是数字呢？而是时间，或者一些字符编码呢？

In [102]:
df = pl.DataFrame(
    {'Id': ['a','a','b','b','b','b','c','c','xuan','xuan','xuan','wang','wang','wang','风','风','风'],
 'Age': [1,4,1,2,3,4,1,2,1,2,3,1,2,4,2,3,4],
 'Value': [1,142,4,73,109,145,6,72,-8,67,102,-1,72,150,72,111,149]})
df

Id,Age,Value
str,i64,i64
"""a""",1,1
"""a""",4,142
"""b""",1,4
"""b""",2,73
"""b""",3,109
"""b""",4,145
"""c""",1,6
"""c""",2,72
"""xuan""",1,-8


通过cumcount生成组内索引/序号

In [115]:
df.with_column(
    pl.col('Id').over('Id').cumcount().alias('code')
).filter(
    pl.col('Age').rank().over('Id') == pl.col('Age')
).drop('code')
# 最后再把组内索引去掉

Id,Age,Value
str,i64,i64
"""a""",1,1
"""b""",1,4
"""b""",2,73
"""b""",3,109
"""b""",4,145
"""c""",1,6
"""c""",2,72
"""xuan""",1,-8
"""xuan""",2,67


### 组内排序

In [121]:
df = pl.DataFrame({
  'group': [2,2,1,1,2,2,1],
  'value': [3,4,3,1,1,3,6],
  'value2': [5,4,3,2,1,0,5],
})
df

group,value,value2
i64,i64,i64
2,3,5
2,4,4
1,3,3
1,1,2
2,1,1
2,3,0
1,6,5


这是直接按'value','value2'进行联动排序

In [122]:
df.select(
    pl.all().sort_by(['value','value2'])
)

group,value,value2
i64,i64,i64
2,1,1
1,1,2
2,3,0
1,3,3
2,3,5
2,4,4
1,6,5


这是先对group进行分组，再对'value','value2'排序？？  
还是先对'value','value2'排序，再对group分组？这似乎和结果对不上。

In [123]:
df.select(
    pl.all().sort_by(['value','value2']).over('group')
)

group,value,value2
i64,i64,i64
2,1,1
2,3,0
1,1,2
1,3,3
2,3,5
2,4,4
1,6,5


这是先对group进行分组，再对'value','value2'排序，再对结果进行group的排序

In [124]:
df.select(
    pl.all().sort_by(['value','value2']).over('group').sort_by(['group'])
)

group,value,value2
i64,i64,i64
1,1,2
1,3,3
1,6,5
2,1,1
2,3,0
2,3,5
2,4,4


### 从时间戳创建时间格式数据

由于polars的时间戮是从毫秒ms开始的，所以要乘以1000

In [144]:
# 原始数据是从秒开始的
df = pl.DataFrame({
    "epoch_seconds": [1648457740, 1648457740 + 10]
})
df

epoch_seconds
i64
1648457740
1648457750


In [143]:
MILLISECONDS_IN_SECOND = 1000;

df.select(
    (pl.col("epoch_seconds") * MILLISECONDS_IN_SECOND).cast(
        pl.Datetime).dt.with_time_unit("ms").alias("datetime")
)

datetime
datetime
2022-03-28 08:55:40
2022-03-28 08:55:50


### 拼接DF

rechunk the final DataFrame/Series.

In [147]:
df1 = pl.DataFrame({"a": [1], "b": [2], "c": [3]})
df2 = pl.DataFrame({"a": [4], "b": [5], "c": [6]})


# new memory slab
new_df = pl.concat([df1, df2], rechunk=True)
new_df

a,b,c
i64,i64,i64
1,2,3
4,5,6


In [148]:
# append free (no memory copy)
new_df = df1.vstack(df2)
new_df

a,b,c
i64,i64,i64
1,2,3
4,5,6


In [153]:
# try to append in place
df1.extend(df2)
df1

a,b,c
i64,i64,i64
1,2,3
4,5,6
4,5,6
4,5,6
4,5,6
4,5,6


ham,spam,foo
i64,i64,i64
2,11,3
2,22,2
3,33,1


### 在前面添加一列

In [163]:
df = pl.DataFrame({
    "a": [1, 2, 3],
    "b": [True, None, False]
})

df.select([
    pl.lit("foo").alias("z"),
    pl.all()
])

z,a,b
str,i64,bool
"""foo""",1,true
"""foo""",2,null
"""foo""",3,false


### 使用apply函数

这里`np.left_shift(x, 8)`是位运算

In [164]:
import numpy as np
# import polars as pl


df = pl.DataFrame({"col1": [2, 4, 8, 16]}).lazy()
df.with_column(
    pl.col("col1").apply(lambda x: np.left_shift(x, 8).item()).alias("result")
).collect()

col1,result
i64,i64
2,512
4,1024
8,2048
16,4096


如果需要传入两个列  
If you need to pass multiple columns from Polars to the ufunc,   
then use the struct expression with apply.

In [167]:
df = pl.DataFrame({"col1": [2, 4, 8, 16], "shift": [1, 1, 2, 2]})
df

col1,shift
i64,i64
2,1
4,1
8,2
16,2


In [168]:
df.with_column(
    pl.struct(["col1", "shift"])
    .apply(lambda cols: np.left_shift(cols["col1"], cols["shift"]).item())
    .alias("result")
)


col1,shift,result
i64,i64,i64
2,1,4
4,1,8
8,2,32
16,2,64


### join并填充缺失值

In [169]:
df1 = pl.DataFrame({"key": ["a", "b", "d"], "var1": [1, 1, 1]})
df2 = pl.DataFrame({"key": ["a", "b", "c"], "var2": [2, 2, 2]})
df1.join(df2, on="key", how="outer")

key,var1,var2
str,i64,i64
"""a""",1,2
"""b""",1,2
"""c""",null,2
"""d""",1,null


In [171]:
df1.join(df2, on="key", how="outer").with_column(
    pl.all().fill_null(999)
)

key,var1,var2
str,i64,i64
"""a""",1,2
"""b""",1,2
"""c""",999,2
"""d""",1,999


### 设置显示的行数

In [ ]:
pl.Config.set_tbl_rows(1000)

### 加总部分列求和

In [172]:
df = pl.DataFrame(
    {
        "id": [1, 2],
        "cat_a": [2, 7],
        "cat_b": [5, 1],
        "cat_c": [0, 3]
    }
)
df

id,cat_a,cat_b,cat_c
i64,i64,i64,i64
1,2,5,0
2,7,1,3


In [173]:
df.with_column(
    pl.fold(0, lambda acc, s: acc + s, pl.all().exclude("id")).alias("horizontal_sum")
)

id,cat_a,cat_b,cat_c,horizontal_sum
i64,i64,i64,i64,i64
1,2,5,0,7
2,7,1,3,11


先来学习下fold函数

In [177]:
df = pl.DataFrame(
    {
        "a": [2, 1, 3],
        "b": [1, 2, 3],
        "c": [1.0, 2.0, 3.0],
    }
)
df

a,b,c
i64,i64,f64
2,1,1.0
1,2,2.0
3,3,3.0


In [178]:
df.fold(lambda s1, s2: s1 + s2)

shape: (3,)
Series: 'a' [f64]
[
	4.0
	5.0
	9.0
]

In [180]:
df.fold(lambda s1, s2: s1.zip_with(s1 < s2, s2))

shape: (3,)
Series: 'a' [f64]
[
	1.0
	1.0
	3.0
]

### 根据另一个DF的值，对当前DF中某列的值进行修改

当df和df1中的TS列的值想值相等时  
根据用df1中Dr的值替换df中mmsi的值

In [6]:
import polars as pl

In [2]:
df = pl.DataFrame({"TS": [1, 2, 3, 4, 5, 6, 7], "mmsi":[11,12,13,14,15,16,17]})
df

TS,mmsi
i64,i64
1,11
2,12
3,13
4,14
5,15
6,16
7,17


In [3]:
df1 = pl.DataFrame({
    "TS": [4, 6, 7], 
    "Dr": [21,22,23]})
df1

TS,Dr
i64,i64
4,21
6,22
7,23


In [7]:
df.join(
    df1,
    on="TS",
    how="left"
).with_column(
    pl.col('Dr').fill_null( pl.col('mmsi') )
)

TS,mmsi,Dr
i64,i64,i64
1,11,11
2,12,12
3,13,13
4,21,21
5,15,15
6,22,22
7,23,23


In [5]:
df = (
    df.join(df1, on="TS", how="left")
    .with_column(pl.col("Dr").fill_null(pl.col("mmsi")))
    .drop("mmsi")
    .rename({"Dr": "mmsi"})
)
print(df)

shape: (7, 2)
┌─────┬──────┐
│ TS  ┆ mmsi │
│ --- ┆ ---  │
│ i64 ┆ i64  │
╞═════╪══════╡
│ 1   ┆ 11   │
├╌╌╌╌╌┼╌╌╌╌╌╌┤
│ 2   ┆ 12   │
├╌╌╌╌╌┼╌╌╌╌╌╌┤
│ 3   ┆ 13   │
├╌╌╌╌╌┼╌╌╌╌╌╌┤
│ 4   ┆ 21   │
├╌╌╌╌╌┼╌╌╌╌╌╌┤
│ 5   ┆ 15   │
├╌╌╌╌╌┼╌╌╌╌╌╌┤
│ 6   ┆ 22   │
├╌╌╌╌╌┼╌╌╌╌╌╌┤
│ 7   ┆ 23   │
└─────┴──────┘
